# Parameters

In [1]:
num_of_epochs = 5
batch_size = 32
dataset_name = 'plantVillage-test-mohanty'
dataset_path = '../datasets/' + dataset_name
model_diagram_path = 'vgg16-top-removed.png'
model_save_path = 'plantVillage-alexnet-from-scratch.h5'
plot_save_path = 'plot-plantVillage-alexnet-from-scratch.png'

input_width = 224
input_height = 224
input_depth = 3


# Get classes
import os
import re
classes = os.listdir(dataset_path)
class_names = []

for i in classes:
    if(re.search("Tomato___", i)):
        class_names.append(i)
    
print('Classes: ', class_names)
print(len(class_names))


Classes:  ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy']
3


# Initialize base model

In [2]:
import os, sys

from keras.utils.vis_utils import plot_model
from keras.applications import VGG16
from keras.layers import Input

from keras.optimizers import SGD

base_model = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape = (input_width, input_height, input_depth)))

base_model.summary()
plot_model(base_model, to_file=model_diagram_path, show_shapes=True)

Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________

# Attach new head

In [3]:
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.utils.vis_utils import plot_model
from keras.models import Model


# Create head part
head_model = base_model.output

head_model = Flatten(name='flatten')(head_model)
head_model = Dense(4096,activation='relu')(head_model)
head_model = Dense(4096,activation='relu')(head_model)
head_model = Dense(len(class_names),activation='softmax')(head_model)

# Attach head to model
model = Model(inputs=base_model.input, outputs = head_model)

model_diagram_path = 'vgg16-output-modified.png'
plot_model(model, to_file=model_diagram_path, show_shapes=True)


In [4]:
# copile model
opt = SGD(lr=0.05)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# Load dataset

In [5]:
import sys, os
sys.path.append('..')
import numpy as np
from imutils import paths



from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# Get the list of image names
image_paths = list(paths.list_images(dataset_path))

print(len(image_paths), 'images loaded')

# Get unique class_names
class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
class_names = [str(x) for x in np.unique(class_names)]
print(class_names)


# Preprocessors
aap = AspectAwarePreprocessor(input_width, input_height)
itap = ImageToArrayPreprocessor()

# Add preprocessors to DataSetLoader class
sdl = SimpleDatasetLoader(preprocessors=[aap, itap])

(data, labels) = sdl.load(image_paths, verbose=500)
data = data.astype('float') / 255.0



1458 images loaded
['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy']
[INFO]: Processed 500/1458
[INFO]: Processed 1000/1458


# Encode labels and split dataset

In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Split data into training (75%) and testing (25%) data
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
test_size=0.25, random_state=0)

# Convert the labels from integers to vectors
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)

# Fit model

In [7]:
# train the network
print("[INFO] training network...")
H = model.fit(train_x, train_y, validation_data=(test_x, test_y),
              batch_size=batch_size, epochs=num_of_epochs, verbose=1)

model.save(model_save_path)

[INFO] training network...
Instructions for updating:
Use tf.cast instead.
Train on 1093 samples, validate on 365 samples
Epoch 1/5
  64/1093 [>.............................] - ETA: 22:50 - loss: 4.4007 - acc: 0.3438

# Generate classfication report

In [ ]:
from sklearn.metrics import classification_report


# If one hot encoding is used
predictions = model.predict(test_x, batch_size=batch_size)
print(classification_report(test_y.argmax(axis=1), predictions.argmax(axis=1), target_names=class_names))


# Plot model

In [ ]:
print(H.history.keys())

In [ ]:
# plot the training loss and accuracy
from matplotlib import pyplot as plt
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, num_of_epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, num_of_epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, num_of_epochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, num_of_epochs), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(plot_save_path)
plt.show()
